In [3]:
import random as rnd
import copy as cp


class Chromosome:
    def __init__(self, genes_no):
        self._pool = []
        self._generate_pool(genes_no)

        
    def _generate_pool(self, genes_no):
        for i in range(0, genes_no):
            self._pool.append(rnd.randint(0,1))
        
        
    def _get_fitness(self):
        ones = 0
        
        for i in range(0, len(self._pool)):
            if self._pool[i] == 1:
                ones += 1
                
        return ones
    
    
    def get_score(self):
        return self._get_fitness() / len(self._pool)
    
    
    def crossover_with(self, chromosome, points_no):
        if chromosome is None:
            raise AttributeError("The crossover chromosome is absent.")
        
        if points_no >= len(self._pool) or points_no <= 0:
            raise AttributeError("The number of crossover points is improper.")
            
        if len(chromosome._pool) != len(self._pool):
            raise AttributeError("The number of genes between chromosomes differs.")
            
        self._make_crossover(chromosome, points_no)
    
    
    def _get_partition_points(self, points_no):
        # Indices can be the same.
        indices = []
        
        for i in range(0, points_no):
            index = rnd.randint(0, len(self._pool)-1)
            indices.append(index)
            
        return indices
    
    
    def _make_crossover(self, chromosome, points_no):
        indices = self._get_partition_points(points_no)
        
        for i in range(0, len(indices)):
            crossover_index = indices[i]
            from_left = True
            
            if rnd.random() > 0.5:
                from_left = False
                
            #print('from_left=' + str(from_left) + ', crossover_index=' + str(crossover_index))
            #print('lhs=' + str(self._pool) + ',rhs=' + str(chromosome._pool) + ' -> ', end='')
            self._exchange_genes(crossover_index, chromosome, i, from_left)
            #print('lhs=' + str(self._pool) + ',rhs=' + str(chromosome._pool) + '.', end='\n')
    
    
    def _exchange_genes(self, crossover_index, chromosome, j, from_left):
        if from_left == True:    
            for j in range(0, crossover_index):
                self._swap_genes(chromosome, j)
            
            return
        
        for k in range(crossover_index, len(self._pool)):
            self._swap_genes(chromosome, k)

        
    def _swap_genes(self, chromosome, j):
        tmp = self._pool[j]
        self._pool[j] = chromosome._pool[j]
        chromosome._pool[j] = tmp
        
        
    def mutate(self, points_no):
        if points_no > len(self._pool):
            raise AttributeError("Cannot mutate more genes then available.")
        
        indices = self._get_partition_points(points_no)
        #print('indices=' + str(indices) + ', pool=' + str(self._pool), end='')
        
        for i in range(0, len(indices)):
            self._flip_gene(indices[i])
        
        #print(' -> pool=' + str(self._pool))
            
            
    def _flip_gene(self, index):
        if self._pool[index] == 1:
            self._pool[index] = 0
            return
        
        self._pool[index] = 1
        
        
    def to_string(self):
        score = str(int(self.get_score() * 100)) + '%'
        return '[' + str(self._pool) + ',' + score + ']'
    
    
    def get_copy(self):
        chromo = Chromosome(len(self._pool))
        chromo._pool = cp.deepcopy(self._pool)
        return chromo
    
    
    def size(self):
        return len(self._pool)
    

In [162]:
#Chromosome demo
GENES_NO = 6
chromo = Chromosome(GENES_NO)

print('pool=' + str(chromo._pool))
print('fitness=' + str(chromo._get_fitness()))
print('score=' + str(chromo.get_score()))
print()
print('CROSSOVER')
print(chromo.crossover_with(Chromosome(GENES_NO), 3))
print()
print('MUTATION')
print(chromo.mutate(2))


pool=[0, 1, 0, 1, 0, 0]
fitness=2
score=0.3333333333333333

CROSSOVER
from_left=True, crossover_index=1
lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1] -> lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1].
from_left=False, crossover_index=6
lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1] -> lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1].
from_left=True, crossover_index=5
lhs=[0, 1, 0, 1, 0, 0],rhs=[0, 0, 1, 1, 0, 1] -> lhs=[0, 0, 1, 1, 0, 0],rhs=[0, 1, 0, 1, 0, 1].
None

MUTATION
indices=[2, 1], pool=[0, 0, 1, 1, 0, 0] -> pool=[0, 1, 0, 1, 0, 0]
None


In [161]:
import math


class GeneticAlgorithm:
    def __init__(self, **kwargs):
        self._initialize(**kwargs)
        self._chromosomes = []
    
    
    def _initialize(self, **kwargs):
        self._chromosomes_no = int(kwargs["chromosomes_no"])
        self._genes_no = int(kwargs["genes_no"])
        self._generations_no = int(kwargs["generations_no"])
        self._crossover_points = int(kwargs["crossover_points"])
        self._mutation_points = int(kwargs["mutation_points"])
        self._crossover_probability = float(kwargs["crossover_probability"])
        self._mutation_probability = float(kwargs["mutation_probability"])
        self._roulette_wheel_spins_no = int(kwargs["roulette_wheel_spins_no"])
        self._roulette_wheel_type = str(kwargs["roulette_wheel_type"])
    
    
    def _generate_initial_population(self):
        for i in range(0, self._chromosomes_no):
            self._chromosomes.append(Chromosome(self._genes_no))
    
    
    def _select_using_roulette(self):
        normalized = self._get_normalized_scores()
        probabilities = self._spin_the_wheel(normalized)
        offspring = self._create_offspring_as_new_population(probabilities, self._roulette_wheel_spins_no)
        #print('normalized=' + str(normalized))
        #print('probabilities=' + str(probabilities))
        return offspring
        
                
    def _get_normalized_scores(self):
        scores_sum = 0.0
        normalized = []
        
        for i in range(0, len(self._chromosomes)):
            chromo = self._chromosomes[i]
            scores_sum += chromo.get_score()
            
        for j in range(0, len(self._chromosomes)):
            chromo = self._chromosomes[j]
            normalized.append(chromo.get_score() / scores_sum)
            
        return normalized
    
    
    def _spin_the_wheel(self, normalized):
        if self._roulette_wheel_type == str("sum"):
            probabilities = [sum(normalized[0:i+1]) for i in range(len(normalized))]
        elif self._roulette_wheel_type == str("square"):
            probabilities = [normalized[i]*normalized[i] for i in range(len(normalized))]
            probabilities = self._normalize(probabilities)
        elif self._roulette_wheel_type == str("logarithm"):
            probabilities = [math.log(normalized[i]) for i in range(len(normalized))]
            probabilities = self._normalize(probabilities)
            
        return probabilities
    
    
    def _normalize(self, collection):
        denominator = sum(collection)
        normed_collection = [collection[i]/denominator for i in range(len(collection))]
        return normed_collection
    
    
    def _create_offspring_as_new_population(self, probabilities, number_of):
        offspring = []
        
        for i in range(0, number_of):
            r = rnd.random()
            
            for (j,chromosome) in enumerate(self._chromosomes):
                if r <= probabilities[j]:
                    offspring.append(chromosome)
                    break
                    
        return offspring
          
    
    def perform(self):
        print("START")
        self._generate_initial_population()
        iterations = 0
        
        while iterations < self._generations_no:
            offspring = self._select_using_roulette()
            
            for i in range(0, len(offspring)):
                individual = offspring[i]
                
                if rnd.random() < self._crossover_probability:
                    index_1 = rnd.randint(0, len(offspring) - 1)
                    index_2 = rnd.randint(0, len(offspring) - 1)
                    individual_1 = offspring[index_1]
                    individual_2 = offspring[index_2]
                    individual_1.crossover_with(individual_2, self._crossover_points)
                    
                if rnd.random() < self._mutation_probability:
                    individual.mutate(self._mutation_points)
            
            self._chromosomes = offspring
            iterations += 1
            
            #print('offspring=', end='')
            #for i in range(0, len(offspring)):
            #   print(offspring[i].to_string(), end='')
            #print('\n\n')
            
        print("END")
        
    

In [115]:
#GeneticAlgorithm demo

params = {
    "chromosomes_no" : 5,                # population size
    "genes_no" : 7,                      # chromosome length
    "generations_no" : 4,                # number of iterations (converging steps)
    "crossover_points" : 2,
    "mutation_points" : 3,
    "crossover_probability" : 0.5,
    "mutation_probability" : 0.25,
    "roulette_wheel_type" : "sum",       # [sum, square, logarithm]
    "roulette_wheel_spins_no" : 3,
}

ga = GeneticAlgorithm(**params)
ga.perform()


START
normalized=[0.19047619047619047, 0.2857142857142857, 0.09523809523809523, 0.2380952380952381, 0.19047619047619047]
probabilities=[0.19047619047619047, 0.47619047619047616, 0.5714285714285714, 0.8095238095238095, 1.0]
offspring=[[0, 1, 0, 1, 1, 1, 1],71%][[1, 1, 1, 1, 0, 1, 1],85%][[0, 0, 0, 1, 1, 1, 0],42%]


normalized=[0.35714285714285715, 0.42857142857142855, 0.21428571428571427]
probabilities=[0.35714285714285715, 0.7857142857142857, 1.0]
offspring=[[0, 0, 0, 1, 0, 0, 0],14%][[0, 0, 0, 1, 0, 0, 0],14%][[0, 1, 0, 1, 1, 1, 0],57%]


normalized=[0.16666666666666666, 0.16666666666666666, 0.6666666666666666]
probabilities=[0.16666666666666666, 0.3333333333333333, 1.0]
offspring=[[1, 1, 0, 1, 1, 0, 1],71%][[0, 0, 0, 1, 0, 0, 0],14%][[1, 1, 0, 1, 1, 0, 1],71%]


normalized=[0.4545454545454545, 0.0909090909090909, 0.4545454545454545]
probabilities=[0.4545454545454545, 0.5454545454545454, 0.9999999999999999]
offspring=[[1, 0, 1, 1, 1, 1, 1],85%][[1, 0, 1, 1, 1, 1, 1],85%][[1, 0, 1, 1,

In [234]:
params = {
    "chromosomes_no" : 1000,
    "genes_no" : 30,
    "generations_no" : 11,
    "crossover_points" : 1,
    "mutation_points" : 3,
    "crossover_probability" : 0.5,
    "mutation_probability" : 0.25,
    "roulette_wheel_type" : "sum",
    "roulette_wheel_spins_no" : 20,
}

ga = GeneticAlgorithm(**params)

for j in range(0, 2):
    ga.perform()

    for i in range(len(ga._chromosomes)):
        print(ga._chromosomes[i].to_string())


START
END
[[1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],46%]
[[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],66%]
[[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],66%]
[[1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0],50%]
[[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],66%]
[[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],66%]
[[1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0],50%]
[[1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],46%]
[[0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1],53%]
[[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],66%]
[[0, 0, 1, 1, 0, 1, 

In [232]:
print()
